<h1> tf.estimatorを利用</h1>

このnotebookでは、tf.estimatorを用いて機械学習のモデルを作成し、そのパフォーマンスを評価します。<br>
データセットは小さいもの(7700サンプル）なので、すべてインメモリで行います。生のデータをそのままで読み込み処理します。


In [ ]:
import datalab.bigquery as bq
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil

print(tf.__version__)

前の章で作成したデータを読み込みます。

In [ ]:
# CSVでは、1行目にラベルが格納されています。特徴量のあとに、キーが続きます。
CSV_COLUMNS = ['fare_amount', 'pickuplon','pickuplat','dropofflon','dropofflat','passengers', 'key']
FEATURES = CSV_COLUMNS[1:len(CSV_COLUMNS) - 1]
LABEL = CSV_COLUMNS[0]

df_train = pd.read_csv('./taxi-train.csv', header = None, names = CSV_COLUMNS)
df_valid = pd.read_csv('./taxi-valid.csv', header = None, names = CSV_COLUMNS)

<h2> Pandas Dataframeからtf.constantに読み込むためのInput関数を作成する</h2>

In [ ]:
def make_input_fn(df, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 1000,
    num_threads = 1
  )

### モデルのための特徴量のカラム(feature columns)を作成する

In [ ]:
def make_feature_cols():
  input_columns = [tf.feature_column.numeric_column(k) for k in FEATURES]
  return input_columns

<h3> tf.Estimatorフレームワークを利用した線形回帰</h3>

In [ ]:
tf.logging.set_verbosity(tf.logging.INFO)

OUTDIR = 'taxi_trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

model = tf.estimator.LinearRegressor(
      feature_columns = make_feature_cols(), model_dir = OUTDIR)

model.train(input_fn = make_input_fn(df_train, num_epochs = 10))

検証データを用いて評価を行います（テストデータは、最終的なモデルができる段階まで使うべきではありません）

In [ ]:
def print_rmse(model, name, df):
  metrics = model.evaluate(input_fn = make_input_fn(df, 1))
  print('RMSE on {} dataset = {}'.format(name, np.sqrt(metrics['average_loss'])))
print_rmse(model, 'validation', df_valid)

これは、私たちの定めたベンチマーク（同データセットで6ドル程度のRMSE）には程遠い結果ですが、TensorFlowコードの挙動を理解することができましたか？<br>
では、このモデルを予測に使ってみましょう。

In [ ]:
import itertools
# 保存したモデルを読み込んで予測に利用
model = tf.estimator.LinearRegressor(
      feature_columns = make_feature_cols(), model_dir = OUTDIR)
preds_iter = model.predict(input_fn = make_input_fn(df_valid, 1))
print([pred['predictions'][0] for pred in list(itertools.islice(preds_iter, 5))])

RMSEがなぜ高いのかがわかりました。このモデルはすべての乗車に対して同じ金額を予想しています。<br>
さらに複雑なモデルを使うべきでしょうか？では、Deep Neural Networkを使ってみましょう。とても簡単に実装することができます。

<h3> Deep Neural Networkでの回帰 </h3>

In [ ]:
tf.logging.set_verbosity(tf.logging.INFO)
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
model = tf.estimator.DNNRegressor(hidden_units = [32, 8, 2],
      feature_columns = make_feature_cols(), model_dir = OUTDIR)
model.train(input_fn = make_input_fn(df_train, num_epochs = 100));
print_rmse(model, 'validation', df_valid)

どちらのモデルでもベンチマークを超えることはできませんでした。何が起こっているのでしょうか？<br>
私たちは機械学習を行うためにTensorFlowを使っていますが、まだ十分に活用できていません。このコースの残りの時間で、その方法について学んでいきましょう！

では、記録のために、モデルの誤差が少ない方を選んでRMSEを計算しましょう。ここでようやく、テストデータを使用します。

<h2> ベンチマークデータセット </h2>

ベンチマークデータセットを使用してみましょう

In [ ]:
import datalab.bigquery as bq
import numpy as np
import pandas as pd

def create_query(phase, EVERY_N):
  """
  phase: 1 = train 2 = valid
  """
  base_query = """
SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  CONCAT(STRING(pickup_datetime), STRING(pickup_longitude), STRING(pickup_latitude), STRING(dropoff_latitude), STRING(dropoff_longitude)) AS key,
  DAYOFWEEK(pickup_datetime)*1.0 AS dayofweek,
  HOUR(pickup_datetime)*1.0 AS hourofday,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count*1.0 AS passengers,
FROM
  [nyc-tlc:yellow.trips]
WHERE
  trip_distance > 0
  AND fare_amount >= 2.5
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  AND passenger_count > 0
  """

  if EVERY_N == None:
    if phase < 2:
      # Training
      query = "{0} AND ABS(HASH(pickup_datetime)) % 4 < 2".format(base_query)
    else:
      # Validation
      query = "{0} AND ABS(HASH(pickup_datetime)) % 4 == {1}".format(base_query, phase)
  else:
    query = "{0} AND ABS(HASH(pickup_datetime)) % {1} == {2}".format(base_query, EVERY_N, phase)
    
  return query

query = create_query(2, 100000)
df = bq.Query(query).to_dataframe()

In [ ]:
print_rmse(model, 'benchmark', df)

データセットを用いたRMSEは<b>9.41</b>です。（ランダムシードの影響で値は前後します）

6.00ドルという私たちのベンチマークには程遠いですし、距離ベースのルールモデルのRMSE（8.02ドル）も超えられていません。

ですが、怖気づくことはありません。私たちはTensorFlowのモデルをどのように書くかを学びましたが、まだ、機械学習モデルのパフォーマンス向上のために必要なことをしていません。<br>
それは次の章で扱っていきましょう。その際、TensorFlowのモデルをそうした改善に適用していきます。


Copyright 2017 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License